In [89]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [90]:
ais_test = pd.read_csv('ais_test.csv')
ais_train = pd.read_csv('first_50000_rows.csv', sep='|')


# Merge test set with training set history
ais_train['time'] = pd.to_datetime(ais_train['time'])  # Ensure time is datetime
ais_test['time'] = pd.to_datetime(ais_test['time'])
# Sort by vesselId and time
ais_train = ais_train.sort_values(['vesselId', 'time'])
ais_test = ais_test.sort_values(['vesselId', 'time'])

# Concatenate both dataframes in order to add correct time series features

# label where data comes from before concatenation
ais_train['dataset'] = 'train'
ais_test['dataset'] = 'test'
combined_df = pd.concat([ais_train, ais_test], ignore_index=True)
# Sort by vesselId and time to ensure proper order for calculating lags

# Sort by vesselId and time
combined_df = combined_df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)


In [91]:
    
lag_windows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for lag in lag_windows:
    # Create lag features for latitude and longitude
    combined_df[f'latitude_lag{lag}'] = combined_df.groupby('vesselId')['latitude'].shift(lag)
    combined_df[f'longitude_lag{lag}'] = combined_df.groupby('vesselId')['longitude'].shift(lag)

# Forward fill missing values within each 'vesselId' after lagging
combined_df[[f'latitude_lag{lag}' for lag in lag_windows]] = combined_df.groupby('vesselId')[[f'latitude_lag{lag}' for lag in lag_windows]].fillna(method='ffill')
combined_df[[f'longitude_lag{lag}' for lag in lag_windows]] = combined_df.groupby('vesselId')[[f'longitude_lag{lag}' for lag in lag_windows]].fillna(method='ffill')